In [1]:
import torch
import torchvision
from torch import nn
import os, shutil

from torch.utils.data import DataLoader
# torch.autograd.set_detect_anomaly(True)
import numpy as np
# declare batch size for act function


torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0) 

batch_size = 1148

In [2]:
# GPU Device
gpu_id = 2
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 2: True


In [3]:
home_dir = '/home/kim1'
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
d_dir = home_dir+data_dir

resume = ''

In [4]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [5]:
checkpoint = './log/style1/128/foresic/aug' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)

In [6]:
# Autoencoder part
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        #should change channel 3
        self.conv_1_1 = nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3,padding=1)
        self.relu_1_2 = nn.ReLU(True)
            
        self.conv_2_1 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=2,padding=1)
        self.bn_2_2 = nn.BatchNorm2d(16)
        self.relu_2_3 = nn.ReLU(True)
        
        self.conv_3_1 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=2,padding=1)
        self.bn_3_2 = nn.BatchNorm2d(32)
        self.relu_3_3 = nn.ReLU(True)
        
        self.conv_4_1=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2,padding=1)
        self.bn_4_2=nn.BatchNorm2d(64)
        self.relu_4_3=nn.ReLU(True)
        
        self.conv_5_1 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=2,padding=1)
        self.bn_5_2 =nn.BatchNorm2d(128)
        self.relu_5_3 = nn.ReLU(True)
        
        ## decoder
        self.upsample_6_1 = nn.Upsample(scale_factor=2,mode='nearest')
        self.convtranspose_6_2 = nn.Conv2d(in_channels=128,out_channels=64,kernel_size=3,padding=1)
        self.bn_6_3 = nn.BatchNorm2d(64)
        self.relu_6_4 = nn.ReLU(True)
        
        self.upsample_7_1 = nn.Upsample(scale_factor=2,mode='nearest')
        self.convtranspose_7_2 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,padding=1)
        self.bn_7_3 = nn.BatchNorm2d(32)
        self.relu_7_4 = nn.ReLU(True)
        
        self.upsample_8_1 = nn.Upsample(scale_factor=2,mode='nearest')
        self.convtranspose_8_2 = nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,padding=1)
        self.bn_8_3 = nn.BatchNorm2d(16)
        self.relu_8_4 = nn.ReLU(True)
        
        self.upsample_9_1 = nn.Upsample(scale_factor=2,mode='nearest')
        self.convtranspose_9_2 = nn.Conv2d(in_channels=16,out_channels=8,kernel_size=3,padding=1)
        self.bn_9_3 = nn.BatchNorm2d(8)
        self.relu_9_4 = nn.ReLU(True)
        #should change channel 6
        # 혹시 안되면
        # self.conv_10_1 = nn.ConvTranspose2d(in_channels=8,out_channels=3,kernel_size=3,padding=1)
        #이걸로 마지막 conv 계층을 바꿔주세요 
        self.conv_10_1 = nn.Conv2d(in_channels=8,out_channels=3,kernel_size=3,padding=1)
        self.tanh_10_2=nn.Tanh()
        
        # no masking no *0 or assign value zero not done .        
        
    def forward(self, x,label):
        x1 = self.conv_1_1(x)
        x2 = self.relu_1_2(x1)

        x3 = self.conv_2_1(x2)
        x4 = self.bn_2_2(x3)
        x5 = self.relu_2_3(x4)

        
        x6 = self.conv_3_1(x5)
        x7 = self.bn_3_2(x6)
        x8 = self.relu_3_3(x7)

        
        x9 = self.conv_4_1(x8)
        x10 = self.bn_4_2(x9)
        x11 = self.relu_4_3(x10)

        
        x12 = self.conv_5_1(x11)
        x13 = self.bn_5_2(x12)
        x14 = self.relu_5_3(x13)
        
        act = x14.clone()
        dep = x14.clone()

        # Selection block setting zero values based on label
        # [:64] -> fake data latent space 
        # [64:] -> real data latent space
        # 0->fake 1 ->real
        # 15 15
        A = torch.nn.Parameter(torch.zeros(64,8,8))
        for i in range(len(label)):
            #real 
            if label[i].item():
                #setting fake latent space into zero
                dep[i,:64] = A
            else:
                dep[i,64:]=A
                
        x15 = self.upsample_6_1(dep) 
        x16 = self.convtranspose_6_2(x15)
        x17 = self.bn_6_3(x16)
        x18 = self.relu_6_4(x17) 
        x19 = self.upsample_7_1(x18)
        x20 = self.convtranspose_7_2(x19)
        x21 = self.bn_7_3(x20)
        x22 = self.relu_7_4(x21)
        x23 = self.upsample_8_1(x22)
        x24 = self.convtranspose_8_2(x23)
        x25 = self.bn_8_3(x24)
        x26 = self.relu_8_4(x25)
        x27 = self.upsample_9_1(x26)
        x28 = self.convtranspose_9_2(x27) 
        x29 = self.bn_9_3(x28)
        x30 = self.relu_9_4(x29)
        x31 = self.conv_10_1(x30)
        x32 = self.tanh_10_2(x31)

        return  x32 , act

In [7]:
model = Autoencoder()
model.cuda()

Autoencoder(
  (conv_1_1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_1_2): ReLU(inplace=True)
  (conv_2_1): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn_2_2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_2_3): ReLU(inplace=True)
  (conv_3_1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn_3_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_3_3): ReLU(inplace=True)
  (conv_4_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn_4_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_4_3): ReLU(inplace=True)
  (conv_5_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn_5_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_5_3): ReLU(inplace=True)
  (upsample_6_1): Upsample(scale_factor=2.0, mode=neare

In [8]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-7)

#configuration

num_epochs = 100
criterion1 = nn.L1Loss()

In [9]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

train_dir = os.path.join(d_dir, 'train')
val_dir = os.path.join(d_dir, 'validation')

train_data =torchvision.datasets.ImageFolder(root=train_dir,
                                                transform = transforms.Compose([transforms.Resize((128 ,128 )),transforms.ToTensor()]))
validation_dataset =torchvision.datasets.ImageFolder(root=val_dir,
                                                transform = transforms.Compose([transforms.Resize((128 ,128 )),transforms.ToTensor()]))

In [10]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_data,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=8)

validation_dataloader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=8)

In [11]:
print("train_dataloader.classes : %s" % train_data.classes)
print("validation_dataloader.classes : %s" % validation_dataset.classes)
print("train_dataloader.classes : %s" % train_data.class_to_idx)
print("validation_dataloader.classes : %s" % validation_dataset.class_to_idx)

train_dataloader.classes : ['0', '1']
validation_dataloader.classes : ['0', '1']
train_dataloader.classes : {'0': 0, '1': 1}
validation_dataloader.classes : {'0': 0, '1': 1}


In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = 0
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])

In [13]:
def act_loss_func(outputs, labels):
    batch_size = outputs.size()[0]
    loss_list = torch.zeros([batch_size])
    loss_list = loss_list.to(device)
    for i in range(batch_size):
        #fake
        total_loss =torch.zeros([1],dtype=torch.float32)
        total_loss = total_loss.to(device)
        #real
        total_loss_1 =torch.zeros([1],dtype=torch.float32)
        total_loss_1 = total_loss.to(device)
        # real
        if labels[i].item():
            #fake
            for latent_index in range(64):
                temp= torch.sum(torch.abs(outputs[i,latent_index,:,:]))/225
                total_loss = torch.sum(total_loss+temp)
            #real
            for latent_index in range(64,128):
                temp1= torch.abs(1 -torch.sum(torch.abs(outputs[i,latent_index,:,:]))/225)
                total_loss_1 = torch.sum(total_loss_1+temp1)
        #fake
        else:
            #fake
            for latent_index in range(64):
                temp= torch.abs(1- torch.sum(torch.abs(outputs[i,latent_index,:,:]))/225)
                total_loss = torch.sum(total_loss+temp)
            #real
            for latent_index in range(64,128):
                temp1= torch.sum(torch.abs(outputs[i,latent_index,:,:]))/225
                total_loss_1 = torch.sum(total_loss_1+temp1)
        
        loss_list[i]=total_loss+total_loss_1

        
    return torch.sum(loss_list)

#%%

# test
def act_loss_test(outputs):
    batch_size = outputs.size()[0]
    answer = torch.zeros([batch_size,2])
    answer.cuda()
    for i in range(batch_size):
        fake = torch.zeros([1], dtype=torch.float32).to(device)
        real = torch.zeros([1], dtype=torch.float32).to(device)
        # fake latent space
        for latent_index in range(64):
            fake = fake + torch.sum(torch.abs(outputs[i, latent_index]))
        # real latent space
        for latent_index in range(64, 128):
            real = real + torch.sum(torch.abs(outputs[i, latent_index]))


        answer[i][0] = fake.item() / (fake.item() + real.item())
        answer[i][1] = real.item() / (fake.item() + real.item())
        
            
    return answer


In [ ]:
from torch.autograd import Variable
import torchvision.utils as vutils
from sklearn.metrics import classification_report
device = torch.device("cuda")

target_names = ['fake','real']
loss_val =0
for epoch in range(num_epochs):
    model = model.train()
    for _, (x,label) in enumerate(train_dataloader):
        init = x
        init= init.to(device)
        x = x.view(x.size(),-1)
        x = x.to(device)
        optimizer.zero_grad()
        output,act_data = model(x,label)
        rec_loss = criterion1(output, init)
        act_loss = act_loss_func(act_data, label)
        loss = act_loss+0.1*rec_loss
        loss.backward()
        optimizer.step()
        
        if (_+1) % 10 == 0:
            print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
        
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))

#     vutils.save_image(x,'deepfake_%d_real_samples.png' % epoch,normalize=True)
#     vutils.save_image(output,'deepfake_%d_generated_samples.png' % epoch,normalize=True)
    
    model = model.eval()
    
    pred= []
    labels= []
    correct =0
    total =0
    with torch.no_grad():
        loss = 0
        for _, (x,label) in enumerate(validation_dataloader):
            init = x
            init= init.to(device)
            x = x.view(x.size(),-1)
            x = x.to(device)
            a= label.shape[0]
            temp = torch.rand([a])
            output,act_data = model(x,temp)
            outputs  = act_loss_test(act_data)
            
            rec_loss = criterion1(output, init)
            act_loss = act_loss_func(act_data, label)

            loss += act_loss+0.1*rec_loss  
            _, predicted = torch.max(outputs.data, 1)
            pred += predicted.tolist()
            labels += label.tolist()
            correct += (predicted == label).sum().item()
            
            
            
            optimizer.zero_grad()

        print("validation loss is %f" % loss)
        temp =correct/len(validation_dataset)
        print('Test Accuracy %f %%' % temp)
        print(classification_report(labels, pred, target_names=target_names))
        
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }, is_best=True, checkpoint=checkpoint)

epoch [1/100], loss:72420.8906
epoch [1/100], loss:71904.0938
epoch [1/100], loss:70788.3984
epoch [1/100], loss:69171.7734
epoch [1/100], loss:67139.5078
epoch [1/100], loss:63755.9453
epoch [1/100], loss:63755.9453
validation loss is 464347.906250
Test Accuracy 0.601667 %
              precision    recall  f1-score   support

        fake       0.56      0.95      0.71      3900
        real       0.84      0.25      0.39      3900

    accuracy                           0.60      7800
   macro avg       0.70      0.60      0.55      7800
weighted avg       0.70      0.60      0.55      7800

epoch [2/100], loss:65361.2031
epoch [2/100], loss:64510.2617
epoch [2/100], loss:63516.9414
epoch [2/100], loss:63154.4766
epoch [2/100], loss:62092.5156
epoch [2/100], loss:59288.5156
epoch [2/100], loss:59288.5156
validation loss is 432121.750000
Test Accuracy 0.548974 %
              precision    recall  f1-score   support

        fake       0.53      1.00      0.69      3900
        real  

epoch [15/100], loss:24898.7969
epoch [15/100], loss:24918.6914
epoch [15/100], loss:24888.8848
epoch [15/100], loss:24935.7930
epoch [15/100], loss:24338.6875
epoch [15/100], loss:23207.0078
epoch [15/100], loss:23207.0078
validation loss is 403129.343750
Test Accuracy 0.539487 %
              precision    recall  f1-score   support

        fake       0.52      1.00      0.68      3900
        real       1.00      0.08      0.15      3900

    accuracy                           0.54      7800
   macro avg       0.76      0.54      0.42      7800
weighted avg       0.76      0.54      0.42      7800

epoch [16/100], loss:22916.5410
epoch [16/100], loss:23053.3848
epoch [16/100], loss:22535.1953
epoch [16/100], loss:22831.4746
epoch [16/100], loss:23036.9082
epoch [16/100], loss:21767.6836
epoch [16/100], loss:21767.6836
validation loss is 422787.718750
Test Accuracy 0.505256 %
              precision    recall  f1-score   support

        fake       0.50      1.00      0.67      3900


/home/kim1/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


epoch [21/100], loss:17411.2598
epoch [21/100], loss:17316.8965
epoch [21/100], loss:17745.0527
epoch [21/100], loss:17096.9824
epoch [21/100], loss:17789.4141
epoch [21/100], loss:16847.8301
epoch [21/100], loss:16847.8301
validation loss is 475030.187500
Test Accuracy 0.500000 %
              precision    recall  f1-score   support

        fake       0.50      1.00      0.67      3900
        real       0.00      0.00      0.00      3900

    accuracy                           0.50      7800
   macro avg       0.25      0.50      0.33      7800
weighted avg       0.25      0.50      0.33      7800

epoch [22/100], loss:16560.7070
epoch [22/100], loss:17270.2871
epoch [22/100], loss:16568.4062
epoch [22/100], loss:17092.1680
epoch [22/100], loss:16711.3965
epoch [22/100], loss:16377.5420
epoch [22/100], loss:16377.5420
validation loss is 386330.187500
Test Accuracy 0.937436 %
              precision    recall  f1-score   support

        fake       0.97      0.90      0.94      3900


In [ ]:
MODEL= Autoencoder()
MODEL.load_state_dict(torch.load("stylegan90epoch_.pth"))
MODEL.cuda()
MODEL.eval()

import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

target_data_1 =torchvision.datasets.ImageFolder(root=r"C:\Users\jonathan\Desktop\prj\gan_detection\PGGAN_128\fine_tune",
                                                transform = transforms.Compose([transforms.Resize((128 ,128 )),transforms.ToTensor()]))
target_data_2 =torchvision.datasets.ImageFolder(root=r"C:\Users\jonathan\Desktop\prj\gan_detection\StyleGAN2_256",
                                                transform = transforms.Compose([transforms.Resize((128 ,128 )),transforms.ToTensor()]))


zeroshot_data_1 = torch.utils.data.DataLoader(dataset=target_data_1,
                                          batch_size=batch_size,
                                          shuffle=True)

zeroshot_data_2 = torch.utils.data.DataLoader(dataset=target_data_2,
                                          batch_size=batch_size,
                                          shuffle=True)







with torch.no_grad():
    correct= 0
    loss = 0
    pred= []
    labels = []
    for _, (x,label) in enumerate(zeroshot_data_1):
        init = x
        init= init.to(device)
        x = x.view(x.size(),-1)
        x = x.to(device)
        a= label.shape[0]
        temp = torch.rand([a])
        output,act_data = MODEL(x,temp)
        outputs  = act_loss_test(act_data)
        
        rec_loss = criterion1(output, init)
        act_loss = act_loss_func(act_data, label)

        loss += act_loss+0.1*rec_loss  
        _, predicted = torch.max(outputs.data, 1)
        pred += predicted.tolist()
        labels += label.tolist()
        correct += (predicted == label).sum().item()
        
        
        
        optimizer.zero_grad()

    # print("PGGAN_128 loss is %f" % loss)
    temp =correct/len(zeroshot_data_1)
    print('Test Accuracy %f %%' % temp)
    print(classification_report(labels, pred, target_names=target_names))

with torch.no_grad():
    correct= 0
    loss = 0
    pred= []
    labels = []
    for _, (x,label) in enumerate(zeroshot_data_2):
        init = x
        init= init.to(device)
        x = x.view(x.size(),-1)
        x = x.to(device)
        a= label.shape[0]
        temp = torch.rand([a])
        output,act_data = MODEL(x,temp)
        outputs  = act_loss_test(act_data)
        
        rec_loss = criterion1(output, init)
        act_loss = act_loss_func(act_data, label)

        loss += act_loss+0.1*rec_loss  
        _, predicted = torch.max(outputs.data, 1)
        pred += predicted.tolist()
        labels += label.tolist()
        correct += (predicted == label).sum().item()
        
        
        
        optimizer.zero_grad()

    print("StyleGAN2 loss is %f" % loss)
    temp =correct/len(target_data_2)
    print('Test Accuracy %f %%' % temp)
    print(classification_report(labels, pred, target_names=target_names))